In [ ]:
!pip install transformers
!pip install pytorch_lightning
!pip install loguru

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 28.7 MB/s 
     |████████████████████████████████| 596 kB 56.0 MB/s 
     |████████████████████████████████| 6.6 MB 59.6 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 584 kB 26.3 MB/s 
     |████████████████████████████████| 140 kB 67.0 MB/s 
     |████████████████████████████████| 409 kB 13.0 MB/s 
     |████████████████████████████████| 1.1 MB 59.2 MB/s 
     |████████████████████████████████| 144 kB 89.7 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 271 kB 64.9 MB/s 
Looking in indexes:

In [1]:
import glob
import os
import pandas as pd
import json
import random
import numpy as np

from scripts.train import *
from scripts.compute_embeddings import *
from scripts.evaluate import evaluate

ModuleNotFoundError: ignored

#Data loading

In [2]:
!gdown --id 11TgUL-3ZxM80W202ziC99nsz8D0im20S
with open("data_for_both_mods.json", "r") as file_:
          full_data  = json.load(file_)
random.shuffle(full_data)

qid_to_ind={}
ind=0
for key in full_data:
  if qid_to_ind.get(key['mention_id']) is None:
      qid_to_ind[key['mention_id']]=ind
      ind+=1

test_size=0.15
val_size=0.15

all_ids = np.arange(len(full_data))
test_indexes = random.sample(list(all_ids), int(len(all_ids)*test_size))
train_val_indexes = set(all_ids)-set(test_indexes)
val_indexes = random.sample(list(train_val_indexes), int(len(train_val_indexes)*val_size))
train_indexes = set(train_val_indexes)-set(val_indexes)

assert len(full_data) == len(test_indexes)+len(val_indexes)+len(train_indexes)

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=11TgUL-3ZxM80W202ziC99nsz8D0im20S
To: /content/data_for_both_mods.json
100% 433M/433M [00:04<00:00, 104MB/s]


In [ ]:
#MENTIONS
!mkdir -p data

#test-------------------------------
values = [full_data[ind] for ind in test_indexes]
keys=np.arange(len(test_indexes)).tolist()
test_dict = dict(zip(keys, values))

json_data = json.dumps(test_dict, indent=2, ensure_ascii=False)
with open('data/mentions_test.json', 'w',encoding='utf8') as fp:
    fp.write(json_data)

#val--------------------------------
values = [full_data[ind] for ind in val_indexes]
keys=np.arange(len(val_indexes)).tolist()
val_dict = dict(zip(keys, values))

json_data = json.dumps(val_dict, indent=2, ensure_ascii=False)
with open('data/mentions_val.json', 'w',encoding='utf8') as fp:
    fp.write(json_data)

#train----------------------------
values = [full_data[ind] for ind in train_indexes]
keys=np.arange(len(train_indexes)).tolist()
train_dict = dict(zip(keys, values))

json_data = json.dumps(train_dict, indent=2, ensure_ascii=False)
with open('data/mentions_train.json', 'w',encoding='utf8') as fp:
    fp.write(json_data)

In [ ]:
#ENTITIES
!mkdir -p entities_data

#test-------------------------------
values = [{'entity_id':full_data[ind]['mention_id'],'entity_context':full_data[ind]['entity_context']} for ind in test_indexes]
keys=np.arange(len(test_indexes)).tolist()
test_dict = dict(zip(keys, values))

json_data = json.dumps(test_dict, indent=2, ensure_ascii=False)
with open('entities_data/entities_test.json', 'w',encoding='utf8') as fp:
    fp.write(json_data)

#val--------------------------------
values = [{'entity_id':full_data[ind]['mention_id'],'entity_context':full_data[ind]['entity_context']} for ind in val_indexes]
keys=np.arange(len(val_indexes)).tolist()
val_dict = dict(zip(keys, values))

json_data = json.dumps(val_dict, indent=2, ensure_ascii=False)
with open('entities_data/entities_val.json', 'w',encoding='utf8') as fp:
    fp.write(json_data) 

#Train

In [ ]:
model, trainer = train('./data',batch_size=8,model_type='E',mapping = qid_to_ind,max_epochs=2)
# model, trainer = train('./data',batch_size=8,model_type='F',max_epochs=2)

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Training examples: 51283
Validation examples: 9049


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./logs/MEL
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                       | Type           | Params
--------------------------------------------------------------
0 | mention_embedding_part     | BertEmbeddings | 92.2 M
1 | mention_main_part_4_layers | ModuleList     | 28.4 M
2 | entity_main_part_4_layers  | ModuleList     | 28.4 M
3 | entity_embedding_part      | Embedding      | 21.7 M
4 | fc_me                      | Linear         | 98.4 K
5 | fc_ee                      | Linear         | 98.4 K
--------------------------------------------------------------
170 M     Trainable params
0         Non-trainable params
170 M     Total params
683.097   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

#Evaluate

In [ ]:
#load checkpoints

!mkdir -p check_E
!gdown 1-0iPBs6X5iXIBmYpY4bewvs6dvCFOy9q
!mv epoch=1-val_loss=2.010.ckpt check_E/

!mkdir -p check_F
!gdown 1-4DO5-Mg6PJ9dkuDTNs3gzt1QQUgeQjn
!mv epoch=1-val_loss=0.398.ckpt check_F/

Downloading...
From: https://drive.google.com/uc?id=1-0iPBs6X5iXIBmYpY4bewvs6dvCFOy9q
To: /content/epoch=1-val_loss=2.010.ckpt
100% 2.05G/2.05G [00:18<00:00, 108MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1-4DO5-Mg6PJ9dkuDTNs3gzt1QQUgeQjn
To: /content/epoch=1-val_loss=0.398.ckpt
100% 2.90G/2.90G [00:40<00:00, 72.2MB/s]


In [ ]:
!mkdir -p emb_modE

compute_embeddings(checkpoint_dir='check_E', 
                   data_dir='data', 
                   model_type='E', 
                   batch_size=8, 
                   mapping=qid_to_ind)

!mv mewsli_mention_embeddings_val.npy emb_modE/
!mv mewsli_mention_embeddings_test.npy emb_modE/
!mv mewsli_entity_embeddings_val.npy emb_modE/
!mv mewsli_entity_embeddings_test.npy emb_modE/

Mentions
val
Num mentions: 9049
test
Num mentions: 10646

Entities
val
Num mentions: 9049
test
Num mentions: 10646


In [ ]:
!mkdir -p emb_modF

compute_embeddings(checkpoint_dir='check_F', 
                   data_dir='data', 
                   model_type='F', 
                   batch_size=8, 
                   mapping=None)

!mv mewsli_mention_embeddings_val.npy emb_modF/
!mv mewsli_mention_embeddings_test.npy emb_modF/
!mv mewsli_entity_embeddings_val.npy emb_modF/
!mv mewsli_entity_embeddings_test.npy emb_modF/

Mentions
val
Num mentions: 9049
test
Num mentions: 10646

Entities
val
Num mentions: 9049
test
Num mentions: 10646


In [ ]:
print('Accuracy of the top k retrieved entities (R@k)')
print('\nModel E:')
print('on val dataset: ')
evaluate(embedd_dir='emb_modE',split='val')
print('on test dataset: ')
evaluate(embedd_dir='emb_modE',split='test')

print('\nModel F:')
print('on val dataset: ')
evaluate(embedd_dir='emb_modF',split='val')
print('on test dataset: ')
evaluate(embedd_dir='emb_modF',split='test')

Accuracy of the top k retrieved entities (R@k)

Model E:
on val dataset: 
Retrieval rate (k=50): 0.00586 , (expected rate for random = 0.00553)
Retrieval rate (k=100): 0.01138 , (expected rate for random = 0.01105)
on test dataset: 
Retrieval rate (k=50): 0.00498 , (expected rate for random = 0.0047)
Retrieval rate (k=100): 0.01005 , (expected rate for random = 0.00939)

Model F:
on val dataset: 
Retrieval rate (k=50): 0.19936 , (expected rate for random = 0.00553)
Retrieval rate (k=100): 0.31473 , (expected rate for random = 0.01105)
on test dataset: 
Retrieval rate (k=50): 0.17669 , (expected rate for random = 0.0047)
Retrieval rate (k=100): 0.27926 , (expected rate for random = 0.00939)
